In [1]:
!pip install transformers
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

from transformers import BertTokenizer, TFBertModel

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer

import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate
from keras.models import Model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import to_categorical, pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.text import Tokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Movie Research/yelp_reviews.csv")

In [16]:
data['stars'].unique()

array([3., 5., 4., 1., 2.])

In [6]:
temp = data.sample(n = 400000, random_state=42)
df = temp.sample(frac = .1, random_state=42)

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [10]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

In [11]:
df['text'] = df['text'].str.lower()

In [12]:
df['text'] = df['text'].str.replace('[^\w\s]', '')

<ipython-input-12-e7256b50f5e4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', '')


In [13]:
# Apply the remove_stopwords and lemmatize_text functions to the 'text' column
df['text'] = df['text'].apply(remove_stopwords)
df['text'] = df['text'].apply(lemmatize_text)

In [19]:
train_df = df[:27000]
test_df = df[27000:]

In [20]:
max_features = 20000

In [21]:
# Tokenization
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_df['text'].values))
X_train = tokenizer.texts_to_sequences(train_df['text'].values)
X_test = tokenizer.texts_to_sequences(test_df['text'].values)

In [22]:
x_train = pad_sequences(X_train, maxlen=200)
x_test = pad_sequences(X_test, maxlen=200)

In [23]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

In [24]:
embedding_matrix = np.zeros((nb_words, 200))

In [25]:
embedding_file = '/content/drive/MyDrive/Movie Research/glove.twitter.27B.200d.txt'

# read in embeddings
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embedding_file, 'r', encoding = 'utf-8'))

In [26]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

embedding_matrix = np.zeros((nb_words, 200))

In [27]:
missed = []

In [28]:
for word, i in word_index.items():

    if i == max_features:
        break

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        missed.append(word)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

inp = Input(shape = (200,))
x = Embedding(max_features, 200, weights = [embedding_matrix], trainable = True)(inp)
x = SpatialDropout1D(0.5)(x)
x = Bidirectional(LSTM(60, return_sequences=True))(x)
x = Bidirectional(LSTM(60, return_sequences=True))(x)
x = Bidirectional(GRU(60, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
outp = Dense(5, activation = 'sigmoid')(conc)

model = Model(inputs = inp, outputs = outp)
# patience is how many epochs to wait to see if val_loss will improve again.
earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)
checkpoint = ModelCheckpoint(monitor = 'val_loss', save_best_only = True, filepath = 'yelp_lstm_gru_weights.hdf5')
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
y_train = pd.get_dummies(train_df["stars"])

In [30]:
model.fit(x_train, y_train, batch_size = 512, epochs = 10, validation_split = .1,
          callbacks=[earlystop, checkpoint])

Epoch 1/10
48/48 [==============================] - 227s 4s/step - loss: 0.4464 - accuracy: 0.4833 - val_loss: 0.3573 - val_accuracy: 0.5819
Epoch 2/10
48/48 [==============================] - 213s 4s/step - loss: 0.3443 - accuracy: 0.5888 - val_loss: 0.3122 - val_accuracy: 0.6381
Epoch 3/10
48/48 [==============================] - 213s 4s/step - loss: 0.3113 - accuracy: 0.6274 - val_loss: 0.2921 - val_accuracy: 0.6552
Epoch 4/10
48/48 [==============================] - 215s 4s/step - loss: 0.2944 - accuracy: 0.6461 - val_loss: 0.2829 - val_accuracy: 0.6604
Epoch 5/10
48/48 [==============================] - 215s 4s/step - loss: 0.2827 - accuracy: 0.6637 - val_loss: 0.2770 - val_accuracy: 0.6652
Epoch 6/10
48/48 [==============================] - 216s 5s/step - loss: 0.2724 - accuracy: 0.6776 - val_loss: 0.2761 - val_accuracy: 0.6659
Epoch 7/10
48/48 [==============================] - 214s 4s/step - loss: 0.2654 - accuracy: 0.6869 - val_loss: 0.2735 - val_accuracy: 0.6707
Epoch 8/10
48

In [31]:
model.save('saved_model/sentiment_analyzer')

In [32]:
store = tf.keras.models.load_model('saved_model/sentiment_analyzer')
store.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 200, 200)     4000000     ['input_1[0][0]']                
                                                                                                  
 spatial_dropout1d (SpatialDrop  (None, 200, 200)    0           ['embedding[0][0]']              
 out1D)                                                                                           
                                                                                                  
 bidirectional (Bidirectional)  (None, 200, 120)     125280      ['spatial_dropout1d[0][0]']  

In [ ]:
model.summary()